In [1]:
from features_labels_extract import extract_features_and_labels
import torch
from torch.utils.data import Dataset
import numpy as np
from sklearn.metrics import (accuracy_score, classification_report, confusion_matrix, 
                             hamming_loss, ConfusionMatrixDisplay, 
                             precision_score, recall_score, f1_score, roc_curve, auc)
import yfinance as yf

In [2]:
tickers = [
    # Technology
    "AAPL",  # Apple Inc.
    "MSFT",  # Microsoft Corporation
    "GOOGL", # Alphabet Inc. (Google)
    "AMZN",  # Amazon.com Inc.
    "TSLA",  # Tesla Inc.
    "NVDA",  # NVIDIA Corporation
    "META",  # Meta Platforms Inc. (Facebook)
    "AMD",   # Advanced Micro Devices Inc.
    "INTC",  # Intel Corporation
    "CSCO",  # Cisco Systems Inc.

    # Finance
    "BRK-B", # Berkshire Hathaway Inc.
    "JPM",   # JPMorgan Chase & Co.
    "V",     # Visa Inc.
    "MA",    # Mastercard Inc.
    "GS",    # Goldman Sachs Group Inc.

    # Consumer Goods & Retail
    "NFLX",  # Netflix Inc.
    "DIS",   # Walt Disney Co.
    "PG",    # Procter & Gamble Co.
    "PEP",   # PepsiCo Inc.
    "KO",    # The Coca-Cola Company
    "MCD",   # McDonald's Corporation
    "WMT",   # Walmart Inc.
    "TGT",   # Target Corporation
    "NKE",   # Nike Inc.

    # Energy & Industrials
    "XOM",   # Exxon Mobil Corporation
    "CVX",   # Chevron Corporation
    "BA",    # Boeing Co.
    "CAT",   # Caterpillar Inc.

    # Healthcare & Pharmaceuticals
    "UNH",   # UnitedHealth Group Incorporated
    "PFE",   # Pfizer Inc.
    "JNJ",   # Johnson & Johnson
    "LLY",   # Eli Lilly and Co.
    "MRNA",  # Moderna Inc.
]

X=[]
y=[]
for stock in tickers:
    data = yf.download(stock, start="2000-01-01", end="2020-01-01")[["Close", "Volume"]].dropna()
    features,labels=extract_features_and_labels(stock, data, strategy="momentum",feature_window_length=100)
    X.append(features)
    y.append(labels)

X=np.concatenate(X)
y=np.concatenate(y)
print(X.shape)
print(y.shape)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Final dataset size: 484 samples


[*********************100%***********************]  1 of 1 completed


Final dataset size: 484 samples


...

Final dataset size: 484 samples


[*********************100%***********************]  1 of 1 completed


Final dataset size: 484 samples


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Final dataset size: 484 samples
Final dataset size: 7 samples
(14374, 3, 100)
(14374,)


In [3]:
class TradingDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size, padding='same')
        self.bn = nn.BatchNorm1d(out_channels)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))  # Conv -> BN -> ReLU
        x = self.pool(x)  # Max pooling
        x = self.dropout(x)  # Dropout
        return x

class TradingCNN(nn.Module):
    def __init__(self, input_channels, input_length):
        super(TradingCNN, self).__init__()

        # Feature Extraction: 3 CNN layers
        self.conv_layers = nn.Sequential(
            CNNBlock(input_channels, 64),
            CNNBlock(64, 128),
            CNNBlock(128, 256)
        )

        # Fully Connected Layers
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * (input_length // 8), 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1)  # Output: Binary classification
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [5]:
import torch.optim as optim
from torch.utils.data import DataLoader
from ignite.engine import Engine, Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss, Precision, Recall
from ignite.handlers import EarlyStopping, ModelCheckpoint, TerminateOnNan
from ignite.contrib.handlers import ProgressBar

# Create datasets and dataloaders
train_dataset = TradingDataset(X, y)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model
input_channels = X.shape[1]
input_length = X.shape[2]
model = TradingCNN(input_channels, input_length).to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Create the trainer and evaluators
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
def thresholded_output_transform(output):
    y_pred, y = output
    y_pred = torch.round(torch.sigmoid(y_pred))  # Apply sigmoid and round to get binary output
    return y_pred, y
val_metrics = {
    "accuracy": Accuracy(output_transform=thresholded_output_transform),
    "precision": Precision(output_transform=thresholded_output_transform),
    "recall": Recall(output_transform=thresholded_output_transform),
    "loss": Loss(criterion)
}
train_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)
val_evaluator = create_supervised_evaluator(model, metrics=val_metrics, device=device)

# Attach progress bar to the trainer
pbar = ProgressBar(persist=True)
pbar.attach(trainer, output_transform=lambda x: {"loss": x})

# Validation after each epoch
@trainer.on(Events.EPOCH_COMPLETED)
def run_validation(engine):
    val_evaluator.run(train_loader)
    metrics = val_evaluator.state.metrics
    avg_accuracy = metrics['accuracy']
    avg_loss = metrics['loss']
    print(f"\nValidation - Epoch: {engine.state.epoch} | Avg accuracy: {avg_accuracy:.4f} | Avg loss: {avg_loss:.4f}")

# Early stopping and model checkpointing
early_stopping = EarlyStopping(patience=10, score_function=lambda engine: -engine.state.metrics["loss"], trainer=trainer)
val_evaluator.add_event_handler(Events.COMPLETED, early_stopping)
trainer.add_event_handler(Events.ITERATION_COMPLETED, TerminateOnNan())

checkpoint_handler = ModelCheckpoint(dirname='checkpoints', filename_prefix='trading_cnn', n_saved=1, create_dir=True, require_empty=False, score_function=lambda e: -e.state.metrics["loss"], score_name="val_loss", global_step_transform=lambda e, _: e.state.epoch)
val_evaluator.add_event_handler(Events.COMPLETED, checkpoint_handler, {'best_model': model})


c:\Users\piotr\anaconda3\Lib\site-packages\ignite\handlers\tqdm_logger.py:127: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [6]:

# Run the training loop
trainer.run(train_loader, max_epochs=100)

[1/450]   0%|           [00:00<?]


Validation - Epoch: 1 | Avg accuracy: 0.5856 | Avg loss: 0.6922


[1/450]   0%|           [00:00<?]


Validation - Epoch: 2 | Avg accuracy: 0.7055 | Avg loss: 0.5976


[1/450]   0%|           [00:00<?]

...


Validation - Epoch: 73 | Avg accuracy: 0.8651 | Avg loss: 0.3050


[1/450]   0%|           [00:00<?]

2025-03-06 14:45:06,295 ignite.handlers.early_stopping.EarlyStopping INFO: EarlyStopping: Stop training



Validation - Epoch: 74 | Avg accuracy: 0.8764 | Avg loss: 0.2958


State:
	iteration: 33300
	epoch: 74
	epoch_length: 450
	max_epochs: 100
	output: 0.17700055241584778
	batch: <class 'list'>
	metrics: <class 'dict'>
	dataloader: <class 'torch.utils.data.dataloader.DataLoader'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

In [7]:

X_test=[]
y_test=[]
for stock in tickers:
    data = yf.download(stock, start="2020-01-01", end="2025-01-01")[["Close", "Volume"]].dropna()
    features,labels=extract_features_and_labels(stock, data, strategy="momentum",feature_window_length=100)
    X_test.append(features)
    y_test.append(labels)

X_test=np.concatenate(X_test)
y_test=np.concatenate(y_test)
test_dataset = TradingDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Final dataset size: 106 samples



[*********************100%***********************]  1 of 1 completed

Final dataset size: 106 samples



[*********************100%***********************]  1 of 1 completed

...

[*********************100%***********************]  1 of 1 completed


Final dataset size: 106 samples


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Final dataset size: 106 samples


Final dataset size: 106 samples


In [8]:
logits = []
labels_list = []

def evaluate_step(engine, batch):
    model.eval()
    with torch.no_grad():
        images, label = batch
        images = images.to(device)
        label = label.to(device)
        outputs = model(images)
        logits.append(outputs.cpu().numpy())
        labels_list.append(label.cpu().numpy())
    return outputs, label

evaluator = Engine(evaluate_step)
evaluator.run(test_loader)

# Concatenate results from test evaluation
logits_array = np.concatenate(logits, axis=0)
labels_array = np.concatenate(labels_list, axis=0)

# Convert logits to probabilities using sigmoid
probs = torch.sigmoid(torch.tensor(logits_array)).numpy()
preds = (probs > 0.5).astype(int)
y_true = labels_array.astype(int)
y_pred = preds.astype(int)

# Print classification report
print("\nClassification Report (Per-Class):")
print(classification_report(y_true, y_pred))


Classification Report (Per-Class):
              precision    recall  f1-score   support

           0       0.73      0.95      0.83      1601
           1       0.95      0.70      0.81      1897

    accuracy                           0.82      3498
   macro avg       0.84      0.83      0.82      3498
weighted avg       0.85      0.82      0.82      3498

